In [1]:
# CSL Paper: Dimensional speech emotion recognition from acoustic and text
# Changelog:
# 2019-09-01: initial version
# 2019-10-06: optimizer MTL parameters with linear search (in progress)
# 2019-12-21: update file for CSL journal
#             this python use following feature: mean, std, silence

import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

#from keras_self_attention import SeqSelfAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from attention_helper import AttentionDecoder
#from read_csv import load_features

import random as rn
import tensorflow as tf

# import gensim
# from gensim.models import Word2Vec
# from gensim.models.keyedvectors import KeyedVectors
import codecs

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# x_text = np.load('../improv_loso/data/x_text.npy')

In [4]:
# x_text.shape

In [5]:
# load data - label
list_path_train = '../improv_loso/data/data_mspin_train.csv'
list_train = pd.read_csv(list_path_train, index_col=None)

# vad_train = [list_train['v'], list_train['a'], list_train['d']]
# vad_train = np.array(vad_train).T

list_path_test = '../improv_loso/data/data_mspin_test.csv'
list_test = pd.read_csv(list_path_test, index_col=None)

## Text processing train
list_file = pd.concat([list_train, list_test])
text = list_file['transcript']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
nb_words = len(word_index) + 1
print('Found %s unique tokens' % len(word_index))

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_text = []
x_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

# x_text = np.load('improv_loso/data/x_text.npy')
# x_train_text = x_text[:5744] 
# x_test_text = x_text[5744:]

# vad = np.load('improv_loso/data/vad.npy')
# vad_train = vad[:5744]
# vad_test = vad[5744:]

EMBEDDING_DIM = 300
# nb_words = 3438
# MAX_SEQUENCE_LENGTH = 300

#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
file_loc =  '../../../dimensionalSM/data/crawl-300d-2M-subword.vec'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

Found 3427 unique tokens
../../../dimensionalSM/data/crawl-300d-2M-subword.vec
G Word embeddings: 2000000
G Null word embeddings: 275


In [6]:
vad = np.load('../improv_loso/data/vad.npy')

In [7]:
# standardization
scaled_feature = False

# set Dropout
do = 0.3

# standardized label
scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad


# Concordance correlation coefficient (CCC)-based loss function
# using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [8]:
# split train/test
split = 5744
earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)

In [9]:
# API model, if use RNN, first two rnn layer must return_sequences=True
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model1 = text_model1()

In [10]:
# main function for for LOSO
def main(time):
    model_1 = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [x_text[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [x_text[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict(x_text[4596:], batch_size=32)
    np.save('5744npy-fasttext/text-fasttext_5744-npy'+str(time)+'.npy',  
             np.array(predict).reshape(3, 2570).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('5744npy-fasttext/text-fasttext_5744.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 92s 20ms/step - loss: 2.5234 - v_loss: 0.7279 - a_loss: 0.8804 - d_loss: 0.9101 - v_ccc: 0.2707 - a_ccc: 0.1176 - d_ccc: 0.0883 - val_loss: 2.6183 - val_v_loss: 0.8307 - val_a_loss: 0.8975 - val_d_loss: 0.8889 - val_v_ccc: 0.1690 - val_a_ccc: 0.1024 - val_d_ccc: 0.1104
Epoch 2/50
4595/4595 [==============================] - 89s 19ms/step - loss: 1.9797 - v_loss: 0.5658 - a_loss: 0.6847 - d_loss: 0.7298 - v_ccc: 0.4342 - a_ccc: 0.3157 - d_ccc: 0.2704 - val_loss: 2.6803 - val_v_loss: 0.8560 - val_a_loss: 0.8950 - val_d_loss: 0.9286 - val_v_ccc: 0.1437 - val_a_ccc: 0.1052 - val_d_ccc: 0.0708
Epoch 3/50
4595/4595 [==============================] - 91s 20ms/step - loss: 1.7508 - v_loss: 0.4888 - a_loss: 0.6103 - d_loss: 0.6518 - v_ccc: 0.5110 - a_ccc: 0.3899 - d_ccc: 0.3483 - val_loss: 2.6401 - val_v_loss: 0

4595/4595 [==============================] - 93s 20ms/step - loss: 2.5054 - v_loss: 0.7229 - a_loss: 0.8755 - d_loss: 0.9056 - v_ccc: 0.2764 - a_ccc: 0.1241 - d_ccc: 0.0940 - val_loss: 2.5793 - val_v_loss: 0.8243 - val_a_loss: 0.8629 - val_d_loss: 0.8910 - val_v_ccc: 0.1754 - val_a_ccc: 0.1370 - val_d_ccc: 0.1084
Epoch 2/50
4595/4595 [==============================] - 89s 19ms/step - loss: 1.9869 - v_loss: 0.5711 - a_loss: 0.6864 - d_loss: 0.7304 - v_ccc: 0.4287 - a_ccc: 0.3142 - d_ccc: 0.2702 - val_loss: 2.7307 - val_v_loss: 0.8104 - val_a_loss: 0.9321 - val_d_loss: 0.9871 - val_v_ccc: 0.1891 - val_a_ccc: 0.0677 - val_d_ccc: 0.0125
Epoch 3/50
4595/4595 [==============================] - 89s 19ms/step - loss: 1.7441 - v_loss: 0.4908 - a_loss: 0.6020 - d_loss: 0.6512 - v_ccc: 0.5094 - a_ccc: 0.3975 - d_ccc: 0.3490 - val_loss: 2.6709 - val_v_loss: 0.8033 - val_a_loss: 0.9010 - val_d_loss: 0.9657 - val_v_ccc: 0.1961 - val_a_ccc: 0.0990 - val_d_ccc: 0.0340
Epoch 4/50
4595/4595 [===========

4595/4595 [==============================] - 87s 19ms/step - loss: 1.7494 - v_loss: 0.4900 - a_loss: 0.6020 - d_loss: 0.6578 - v_ccc: 0.5102 - a_ccc: 0.3980 - d_ccc: 0.3424 - val_loss: 2.7539 - val_v_loss: 0.8454 - val_a_loss: 0.9204 - val_d_loss: 0.9876 - val_v_ccc: 0.1540 - val_a_ccc: 0.0796 - val_d_ccc: 0.0125
Epoch 4/50
4595/4595 [==============================] - 87s 19ms/step - loss: 1.5673 - v_loss: 0.4364 - a_loss: 0.5436 - d_loss: 0.5865 - v_ccc: 0.5631 - a_ccc: 0.4561 - d_ccc: 0.4135 - val_loss: 2.7360 - val_v_loss: 0.8524 - val_a_loss: 0.9276 - val_d_loss: 0.9547 - val_v_ccc: 0.1471 - val_a_ccc: 0.0721 - val_d_ccc: 0.0448
Epoch 5/50
4595/4595 [==============================] - 87s 19ms/step - loss: 1.4314 - v_loss: 0.3938 - a_loss: 0.5030 - d_loss: 0.5370 - v_ccc: 0.6071 - a_ccc: 0.4978 - d_ccc: 0.4636 - val_loss: 2.6841 - val_v_loss: 0.8217 - val_a_loss: 0.9056 - val_d_loss: 0.9563 - val_v_ccc: 0.1777 - val_a_ccc: 0.0944 - val_d_ccc: 0.0438
Epoch 6/50
4595/4595 [===========

1422/1422 [==============================] - 8s 5ms/step
[0.2158093899488449, 0.159428671002388, 0.0821429193019867]
5
Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 93s 20ms/step - loss: 2.4809 - v_loss: 0.7339 - a_loss: 0.8600 - d_loss: 0.8873 - v_ccc: 0.2670 - a_ccc: 0.1403 - d_ccc: 0.1119 - val_loss: 2.6664 - val_v_loss: 0.8161 - val_a_loss: 0.9076 - val_d_loss: 0.9427 - val_v_ccc: 0.1838 - val_a_ccc: 0.0926 - val_d_ccc: 0.0573
Epoch 2/50
4595/4595 [==============================] - 88s 19ms/step - loss: 1.9899 - v_loss: 0.5711 - a_loss: 0.6730 - d_loss: 0.7443 - v_ccc: 0.4279 - a_ccc: 0.3265 - d_ccc: 0.2557 - val_loss: 2.7222 - val_v_loss: 0.8651 - val_a_loss: 0.8996 - val_d_loss: 0.9570 - val_v_ccc: 0.1346 - val_a_ccc: 0.1006 - val_d_ccc: 0.0427
Epoch 3/50
4595/4595 [==============================] - 87s 19ms/step - loss: 1.7679 - v_loss: 0.4968 - a_loss: 0.6120 - d_loss: 0.6594 - v_ccc: 0.5036 - a_ccc: 0.3884 - d_ccc: 0.34

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4595/4595 [==============================] - 87s 19ms/step - loss: 0.9609 - v_loss: 0.2641 - a_loss: 0.3399 - d_loss: 0.3567 - v_ccc: 0.7360 - a_ccc: 0.6599 - d_ccc: 0.6432 - val_loss: 2.7678 - val_v_loss: 0.8790 - val_a_loss: 0.9290 - val_d_loss: 0.9589 - val_v_ccc: 0.1203 - val_a_ccc: 0.0709 - val_d_ccc: 0.0410
Epoch 12/50
1422/1422 [==============================] - 8s 5ms/step
[0.22661450505256653, 0.24216531217098236, 0.05494677647948265]
9
Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 95s 21ms/step - loss: 2.5206 - v_loss: 0.7326 - a_loss: 0.8842 - d_loss: 0.9039 - v_ccc: 0.2663 - a_ccc: 0.1160 - d_ccc: 0.0971 - val_loss: 2.6090 - val_v_loss: 0.8254 - val_a_loss: 0.8784 - val_d_loss: 0.9048 - val_v_ccc: 0.1741 - val_a_ccc: 0.1219 - val_d_ccc: 0.0950
Epoch 2/50
4595/4595 [==============================] - 88s 19ms/step - loss: 2.0066 - v_loss: 0.5669 - a_loss: 0.6933 - d_loss: 0.7440 - v_ccc: 0.4333 - a_ccc: 0.3053 - d_ccc:

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4595/4595 [==============================] - 90s 20ms/step - loss: 1.2311 - v_loss: 0.3336 - a_loss: 0.4409 - d_loss: 0.4560 - v_ccc: 0.6661 - a_ccc: 0.5586 - d_ccc: 0.5441 - val_loss: 2.7523 - val_v_loss: 0.8164 - val_a_loss: 0.9297 - val_d_loss: 1.0060 - val_v_ccc: 0.1831 - val_a_ccc: 0.0703 - val_d_ccc: -0.0057
Epoch 8/50
4595/4595 [==============================] - 90s 19ms/step - loss: 1.1402 - v_loss: 0.3101 - a_loss: 0.4094 - d_loss: 0.4200 - v_ccc: 0.6899 - a_ccc: 0.5903 - d_ccc: 0.5796 - val_loss: 2.7622 - val_v_loss: 0.8498 - val_a_loss: 0.9242 - val_d_loss: 0.9881 - val_v_ccc: 0.1497 - val_a_ccc: 0.0760 - val_d_ccc: 0.0122
Epoch 9/50
4595/4595 [==============================] - 88s 19ms/step - loss: 1.0754 - v_loss: 0.2969 - a_loss: 0.3810 - d_loss: 0.3971 - v_ccc: 0.7029 - a_ccc: 0.6188 - d_ccc: 0.6030 - val_loss: 2.7019 - val_v_loss: 0.8441 - val_a_loss: 0.9120 - val_d_loss: 0.9454 - val_v_ccc: 0.1553 - val_a_ccc: 0.0882 - val_d_ccc: 0.0546
Epoch 10/50
4595/4595 [=========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4595/4595 [==============================] - 96s 21ms/step - loss: 2.5288 - v_loss: 0.7225 - a_loss: 0.8881 - d_loss: 0.9164 - v_ccc: 0.2771 - a_ccc: 0.1111 - d_ccc: 0.0829 - val_loss: 2.6058 - val_v_loss: 0.8561 - val_a_loss: 0.8691 - val_d_loss: 0.8803 - val_v_ccc: 0.1438 - val_a_ccc: 0.1311 - val_d_ccc: 0.1194
Epoch 2/50
4595/4595 [==============================] - 90s 20ms/step - loss: 1.9917 - v_loss: 0.5595 - a_loss: 0.6883 - d_loss: 0.7444 - v_ccc: 0.4402 - a_ccc: 0.3120 - d_ccc: 0.2561 - val_loss: 2.6112 - val_v_loss: 0.8574 - val_a_loss: 0.8718 - val_d_loss: 0.8817 - val_v_ccc: 0.1425 - val_a_ccc: 0.1284 - val_d_ccc: 0.1179
Epoch 3/50
4595/4595 [==============================] - 90s 20ms/step - loss: 1.7480 - v_loss: 0.4871 - a_loss: 0.6045 - d_loss: 0.6546 - v_ccc: 0.5123 - a_ccc: 0.3951 - d_ccc: 0.3446 - val_loss: 2.6881 - val_v_loss: 0.8584 - val_a_loss: 0.9054 - val_d_loss: 0.9229 - val_v_ccc: 0.1408 - val_a_ccc: 0.0946 - val_d_ccc: 0.0765
Epoch 4/50
4595/4595 [===========

In [11]:
# hist1 = model1.fit(x_text[:split], vad[:split].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate(x_text[split:], vad[split:].T.tolist())
# print(eval_metrik1[-3:])

# # make prediction
# predict = model1.predict(x_text[4596:], batch_size=32)
# np.save('text_glove_npy_loso', 
#          np.array(predict).reshape(3, 2570).T)